## Make manifest file

This notebook creates `manifest_agu21.csv`, the metadata of the files to be used in the AGU 2021 fall meeting.

In [1]:
import pandas as pd
import glob

In [2]:
df = pd.DataFrame(columns = ['Date', 'Chip (px)', 'Resolution (px)', 'Prefilter', 'Subpixel', 'Software', 'Vx', 'Vy'])

carst_vxs = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/CARST/*/*velo-raw_vx.tif'))
carst_vys = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/CARST/*/*velo-raw_vy.tif'))

for i, j in zip(carst_vxs, carst_vys):
    info = i.split('/')[-2]
    date_raw =  info.split('_')[1]
    chip =      info.split('_')[3][-2:]
    res =       info.split('_')[4][4:]
    prefilter = info.split('_')[5]
    subpixel = '16-node oversampling'
    if date_raw == '20180314':
        date = 'Sen2-20180304-20180314'
    elif date_raw == '20180405':
        date = 'LS8-20180304-20180405'
    elif date_raw == '20180627':
        date = 'Sen2-20180508-20180627'
    elif date_raw == '20180818':
        date = 'LS8-20180802-20180818'
    else:
        raise ValueError('Strange date string!')
    row = {'Date':      date,      'Chip (px)': chip,     'Resolution (px)': res,
           'Prefilter': prefilter, 'Subpixel':  subpixel, 'Software':        'CARST',
           'Vx': i,                'Vy': j}
    df = df.append(row, ignore_index=True)
    
giv_vxs = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/GIV/u*.tif'))
giv_vys = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/GIV/v*.tif'))

for i, j in zip(giv_vxs, giv_vys):
    info = i.split('/')[-1]
    date_raw =  info.split('_')[1]
    chip =      'multi'
    res_raw =   info.split('_')[3][:-4]
    prefilter_raw = info.split('_')[2]
    subpixel = 'interest point groups'
    if date_raw == 's12':
        date = 'Sen2-20180304-20180314'
    elif date_raw == 'l12':
        date = 'LS8-20180304-20180405'
    elif date_raw == 's34':
        date = 'Sen2-20180508-20180627'
    elif date_raw == 'l34':
        date = 'LS8-20180802-20180818'
    else:
        raise ValueError('Strange date string!')
    if res_raw == '50':
        if date_raw == 's12' or date_raw == 's34':
            res = '5'    # 50 m posting (acutal number is 54-40 m)
        elif date_raw == 'l12' or date_raw == 'l34':
            res = '4'    # 60 m posting
        else:
            raise ValueError('Something wrong! #1')
    elif res_raw == '200':
        if date_raw == 's12' or date_raw == 's34':
            res = '20'   # 200 m posting (actual number is 219-160 m)
        elif date_raw == 'l12' or date_raw == 'l34':
            res = '16'   # 240 m posting
        else:
            raise ValueError('Something wrong! #2')
    else:
        raise ValueError('Strange resolution!')
    if prefilter_raw == 'gaus':
        prefilter = 'Gau'
    elif prefilter_raw == 'NAOF':
        prefilter = 'NAOF'
    elif prefilter_raw == 'r':
        prefilter = 'None'
    else:
        raise ValueError('Strange prefilter!')
    row = {'Date':      date,      'Chip (px)': chip,     'Resolution (px)': res,
           'Prefilter': prefilter, 'Subpixel':  subpixel, 'Software':        'GIV',
           'Vx': i,                'Vy': j}
    df = df.append(row, ignore_index=True)
    
vmap_vxs = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/Vmap/**/*vx.tif', recursive=True))
vmap_vys = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/Vmap/**/*vy.tif', recursive=True))

for i, j in zip(vmap_vxs, vmap_vys):
    
    info = i.split('/')[-1]
    res = '1'
    if 'parabolic_subpixel' in i:
        chip = info.split('_')[-3][:2]
        if 'gaussfilter' in i:
            prefilter = 'Gau'
            subpixel = 'parabolic'
        elif 'noprefilter' in i:
            prefilter = 'None'
            subpixel = 'parabolic'
        else:
            raise ValueError('Something wrong! #3')
    elif 'subpixel_comparison' in i:
        prefilter = 'LoG'
        chip = '31'
        if 'spm1' in info:
            subpixel = 'parabolic'
        elif 'spm2' in info:
            subpixel = 'affine adaptive'
        elif 'spm3' in info:
            subpixel = 'affine'
        else:
            raise ValueError('Something wrong! #5')
    else:
        raise ValueError('Something wrong! #4')
    if info.startswith('L'):
        if '20180304' in info:
            date = 'LS8-20180304-20180405'
        elif '20180802' in info:
            date = 'LS8-20180802-20180818'
        else:
            raise ValueError('Something wrong! #6')
    elif info.startswith('T'):
        if '20180304' in info:
            date = 'Sen2-20180304-20180314'
        elif '20180508' in info:
            date = 'Sen2-20180508-20180627'
        else:
            raise ValueError('Something wrong! #7')
    else:
        raise ValueError('Something wrong! #8')
        
    row = {'Date':      date,      'Chip (px)': chip,     'Resolution (px)': res,
           'Prefilter': prefilter, 'Subpixel':  subpixel, 'Software':        'Vmap',
           'Vx': i,                'Vy': j}
    df = df.append(row, ignore_index=True)
    
autorift_vxs = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/autoRIFT/**/*vx.tif', recursive=True))
autorift_vys = sorted(glob.glob('/home/jovyan/Projects/PX_comparison/PX/autoRIFT/**/*vy.tif', recursive=True))

for i, j in zip(autorift_vxs, autorift_vys):
    
    info = i.split('/')[-2]
    subpixel = 'pyrUP'
    res = info.split('_')[-1]
    chip = info.split('_')[-4][:2]
    if 'gauss_hp' in i:
        prefilter = 'Gau'
    elif 'naof2' in i:
        prefilter = 'NAOF'
    elif 's__' in i:
        prefilter = 'None'
    else:
        raise ValueError('Strange prefilter string!')
    if 'Landsat' in i:
        if '20180304' in info:
            date = 'LS8-20180304-20180405'
        elif '20180802' in info:
            date = 'LS8-20180802-20180818'
        else:
            raise ValueError('Strange date string! (autoRIFT) #1')
    elif 'S2' in i:
        if '20180304' in info:
            date = 'Sen2-20180304-20180314'
        elif '20180508' in info:
            date = 'Sen2-20180508-20180627'
        else:
            raise ValueError('Strange date string! (autoRIFT) #2')
    else:
        raise ValueError('Strange date string! (autoRIFT) #3')
    row = {'Date':      date,      'Chip (px)': chip,     'Resolution (px)': res,
           'Prefilter': prefilter, 'Subpixel':  subpixel, 'Software':        'autoRIFT',
           'Vx': i,                'Vy': j}
    df = df.append(row, ignore_index=True)

In [3]:
df

,Date,Chip (px),Resolution (px),Prefilter,Subpixel,Software,Vx,Vy
0,Sen2-20180304-20180314,48,12,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...
1,Sen2-20180304-20180314,48,12,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...
2,Sen2-20180304-20180314,48,12,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...
3,Sen2-20180304-20180314,48,1,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...
4,Sen2-20180304-20180314,48,1,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...
...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,64,8,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...
168,Sen2-20180508-20180627,32,4,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...
169,Sen2-20180508-20180627,32,8,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...
170,Sen2-20180508-20180627,64,4,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...


In [6]:
df.to_csv('manifest_agu21.csv', index=False)